In [1]:
import getpass
user = getpass.getuser()
from omni.isaac.kit import SimulationApp
# Set the path below to your desired nucleus server
simulation_app = SimulationApp({"livesync_usd": f'omniverse://localhost/Users/{user}/isaac/sorting/scratch.usd', "headless": True})
print("Hi")

Passing the following args to the base kit application:  ['-f', '/home/bjnortier/.local/share/jupyter/runtime/kernel-c2523493-50a5-4399-b397-b064fee5f66b.json']
[Warning] [omni.kit.app.plugin] No crash reporter present, dumps uploading isn't available.
[Info] [carb] Logging to file: /home/bjnortier/.local/share/ov/pkg/isaac_sim-2021.2.1/kit/logs/Kit/Isaac-Sim/2021.2/kit_20220218_153219.log
[0.048s] [ext: omni.kit.pipapi-0.0.0] startup
[0.058s] [ext: omni.kit.pip_archive-0.0.0] startup
[0.062s] [ext: omni.usd.config-1.0.0] startup
[0.065s] [ext: omni.usd.libs-1.0.0] startup
[0.237s] [ext: omni.isaac.pip_archive-0.1.0] startup
[0.393s] [ext: omni.stats-0.0.0] startup
[0.395s] [ext: omni.gpu_foundation-0.0.0] startup
[0.401s] [ext: omni.assets.plugins-0.0.0] startup
[0.402s] [ext: omni.kit.loop-isaac-0.1.0] startup
[0.403s] [ext: omni.kit.async_engine-0.0.0] startup
[0.405s] [ext: omni.appwindow-1.0.0] startup
[0.414s] [ext: omni.client-0.1.0] startup
[0.422s] [ext: omni.kit.test-0.0.0] s

[4.098s] [ext: omni.kit.window.preferenSimulation App Starting
Saving a temp livesync stage at  omniverse://localhost/Users/bjnortier/isaac/sorting/scratch.usd  ...Done.
Simulation App Startup Complete
Hi


In [294]:
from omni.isaac.core import World
from pxr import Gf, Sdf, UsdGeom, UsdShade
from omni.isaac.core.utils.prims import create_prim, get_prim_at_path
from omni.isaac.core.utils.rotations import euler_angles_to_quat
from omni.isaac.core.utils.viewports import set_camera_view
from omni.isaac.core.utils.stage import clear_stage
from omni.isaac.core.utils.nucleus import find_nucleus_server
from omni.isaac.core.materials.omni_glass import OmniGlass
from omni.isaac.core.prims.xform_prim import XFormPrim
from omni.isaac.core.utils.extensions import get_extension_path_from_name
from omni.isaac.core.utils.semantics import add_update_semantics
from omni.isaac.core.utils.prims import delete_prim
import omni.isaac.dr as dr
from omni.isaac.synthetic_utils import SyntheticDataHelper     
from pxr import UsdGeom, Usd
from omni.isaac.core.utils.prims import create_prim
from omni.isaac.core.prims.xform_prim import XFormPrim
from omni.isaac.core.utils.rotations import euler_angles_to_quat
from omni.isaac.core.utils.prims import get_prim_path

import omni
import carb
import random
import numpy as np
import torch
from omni.physx.scripts import utils

In [3]:
world = World(stage_units_in_meters = 0.01)
world.step()

In [4]:
dr.commands.ToggleManualModeCommand().do()

In [460]:
CAMERA_DISTANCE = 200

def create_camera():
    camera_rig = UsdGeom.Xformable(create_prim("/World/CameraRig", "Xform"))
    camera = create_prim("/World/CameraRig/Camera", "Camera", position=np.array([0.0, 0.0, CAMERA_DISTANCE]))
    vpi = omni.kit.viewport.get_viewport_interface()
    vpi.get_viewport_window().set_active_camera(str(camera.GetPath()))
    viewport = omni.kit.viewport.get_default_viewport_window()

def create_environment():
    plate = create_prim(
        "/World/Plate",
        "Cube",
        position=np.array([0.0, 0.0, -1.0]),
        scale=np.array([1000.0, 1000.0, 1.0]),
        orientation=euler_angles_to_quat(np.array([0.0, 0.0, 0.0]), degrees=True),
        attributes={"primvars:displayColor": [(71.0 / 255.0, 165.0 / 255.0, 1.0)]},
    )
    utils.setCollider(plate, approximationShape="convexHull")
    
    
def create_light():
    create_prim("/World/LightRig", "Xform")
    create_prim(
        "/World/LightRig/Light",
        "SphereLight",
        position=np.array([0, 0, 200]),
        attributes={"radius": 50, "intensity": 5e4, "color": (1.0, 1.0, 1.0)}
    )
    
def create_dr():
    dr.commands.CreateRotationComponentCommand(
        prim_paths=["/World/LightRig"],
        min_range=(0.0, 15.0, -180.0),
        max_range=(0.0, 60.0, 180.0),
        seed=random.randint(0, 1000)
    ).do()
    dr.commands.CreateRotationComponentCommand(
        prim_paths=["/World/CameraRig"],
        min_range=(0.0, 0.0, -180.0),
        max_range=(0.0, 60.0, 180.0),
        seed=random.randint(0, 1000)
    ).do()
    return


world.clear()
create_camera()
create_environment()
create_light()
create_dr()
world.step()

dr.commands.RandomizeOnceCommand().do()
capture_camera_view()




In [292]:
dr.commands.RandomizeOnceCommand().do()
capture_camera_view()


In [279]:
from omni.isaac.shapenet import utils
cats = ["boat", "plane", "car", "motorbike", "bus"]
categories = [utils.LABEL_TO_SYNSET.get(c, c) for c in cats]
print(categories)

['boat', '02691156', '02958343', 'motorbike', '02924116']


In [252]:
import json
import os
from pathlib import Path
import glob 

shapenet_dir = Path(os.environ["SHAPENET_LOCAL_DIR"])
taxonomy = json.loads((shapenet_dir / "taxonomy.json").read_text())
synset_id_lookup = {}
for entry in taxonomy:
    labels = entry["name"].split(",")
    for label in labels:
        synset_id_lookup[label] = entry["synsetId"]
        


def find_usd_assets(shapenet_dir, categories, max_asset_size=1):
    """Look for USD files under root/category for each category specified.
    For each category, generate a list of all USD files found and select
    assets up to split * len(num_assets) if `train=True`, otherwise select the
    remainder.
    """
    references = {}
    for category in categories:
        all_assets = glob.glob(os.path.join(shapenet_dir, category, "*/*.usd"), recursive=True)
        if max_asset_size is None:
            assets_filtered = all_assets
        else:
            assets_filtered = []
            for a in all_assets:
                assets_filtered.append(a)

        num_assets = len(assets_filtered)
        if num_assets == 0:
            raise ValueError(f"No USDs found for category {category} under max size {max_asset_size} MB.")
        
        def convert_to_nucleus_path(file_path):
            return file_path.replace(
                "/home/bjnortier/shapenet/",
                "omniverse://localhost/Users/bjnortier/isaac/shapenet/",
            )
        
        references[category] = list(map(convert_to_nucleus_path, assets_filtered))
    return references

category_ids = list(map(lambda x: synset_id_lookup[x], ["watercraft", "plane", "car", "bus", "rocket"]))
print(category_ids)

asset_references = find_usd_assets(f"{shapenet_dir}_nomat" , category_ids)


['04530566', '02691156', '02958343', '02924116', '04099429']


In [461]:
from omni.isaac.core.utils.rotations import euler_angles_to_quat

def create_shape_material():
    return PreviewSurface(prim_path="/World/Looks/shape_material", color=np.array([0.2, 0.7, 0.2]))

def add_shape_with_random_pose(category, material):
    ref = random.choice(asset_references[category])
    suffix = str(random.randint(0, 1000))
    print(suffix)
    rig_path=f"/World/Shape/rig{suffix}"
    create_prim(rig_path, "Xform")
    prim_path=f"{rig_path}/mesh{suffix}"
    prim = create_prim(
        prim_path,
        "Xform",
        scale=np.array([20.0, 20.0, 20.0]),
        usd_path=ref,
        semantic_label=category
    )
    
    primX = XFormPrim(prim_path)
    # Random orientation of the object on the ground plane, empirically ShapeNet objects need
    # a 90 degree rotation on the X axis and a random rotation around the Z axis. "euler_angles_to_quat"
    # order seems to be XZY?
    orientation_on_plane = euler_angles_to_quat(np.array([90.0, 0.0, 0.0]), degrees=True)
    primX.set_local_pose(np.array([0.0, 0.0, 0.0]), orientation_on_plane)
    primX.apply_visual_material(material)
    # Determine bounds and translate to sit on the Z=0 plane
    bounds = UsdGeom.Mesh(prim).ComputeWorldBound(0.0, "default")
    new_position = np.array([0.0, 0.0, -bounds.GetBox().GetMin()[2]])
    primX.set_local_pose(new_position)
    
    # Add domain randomization
    dr.commands.CreateMovementComponentCommand(
        prim_paths=[rig_path],
        min_range=(-5.0, -5.0, 0.0),
        max_range=(5.0, 5.0, 0.0),
        seed=random.randint(0, 1000)
    ).do()
    dr.commands.CreateRotationComponentCommand(
        prim_paths=[rig_path],
        min_range=(0.0, 0.0, 0.0),
        max_range=(0.0, 0.0, 360.0),
        seed=random.randint(0, 1000)
    ).do()
    
    
    

In [469]:
   
material = create_shape_material()
add_shape_with_random_pose("04530566", material)
world.step()
world.step()

888


In [171]:
synset_id_lookup["bus"]

'02924116'

In [226]:
from omni.isaac.core.objects import DynamicCuboid

DynamicCuboid(
    prim_path="/World/random_cube",
    name="fancy_cube",
    position=np.array([0, 0, 100.0]),
    size=np.array([50.15, 50.15, 50.15]),
    color=np.array([0, 0, 1.0]),
)
world.step()
world.step()
world.step()

In [482]:
from omni.isaac.core.materials import PreviewSurface

material = PreviewSurface(prim_path="/World/Looks/material1", color=np.array([0.2, 0.7, 1.0]))
plateX = XFormPrim("/World/Plate")
plateX.apply_visual_material(material)

world.step()
world.step()
def capture_camera_view():
    vpi = omni.kit.viewport.get_viewport_interface()
    vpi.get_viewport_window().set_active_camera("/World/CameraRig/Camera")
    viewport = omni.kit.viewport.get_default_viewport_window()
    
    viewport.set_texture_resolution(1024, 1024)
#     print(width, height)

    sd_helper = SyntheticDataHelper()
    gt = sd_helper.get_groundtruth(["rgb"], viewport, verify_sensor_init=True)
    image = gt["rgb"][..., :3]
    # Cast to tensor if numpy array
    if isinstance(gt["rgb"], np.ndarray):
        image = torch.tensor(image, dtype=torch.float, device="cuda")
    # Normalize between 0. and 1. and change order to channel-first.
    image = image.float() / 255.0
    image = image.permute(2, 0, 1)

    from PIL import Image
    im = Image.fromarray(gt["rgb"])
    im.save("rgb.png")
    world.step()
    

dr.commands.RandomizeOnceCommand().do()
capture_camera_view()    